In [1]:
# Install required packages
!pip install transformers datasets evaluate sacrebleu tf-keras ipywidgets sentencepiece sacremoses "accelerate>=0.26.0"
!pip install nltk bert_score wandb trl peft bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 22.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 KB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 KB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 KB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
# Import libraries
import evaluate
import numpy as np
import wandb
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer, 
    EarlyStoppingCallback
)

2025-03-03 21:58:42.415400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741039122.499416    2049 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741039122.523627    2049 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Initialize wandb
wandb.login()  # You'll need to authenticate with your W&B account
wandb.init(project="medical-translation-sft-rl", name="en-fr-medical-translation")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc
wandb: Currently logged in as: enbao (enbao-pok-mon-showdown9713) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# Load dataset
dataset = load_dataset("qanastek/ELRC-Medical-V2", "en-fr")

# Create a proper train/test split to avoid contamination
ds = dataset["train"].train_test_split(test_size=0.2, seed=42) if "train" in dataset else dataset.train_test_split(test_size=0.2, seed=42)

README.md:   0%|          | 0.00/9.04k [00:00<?, ?B/s]

ELRC-Medical-V2.py:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13149 [00:00<?, ? examples/s]

In [5]:
# Further split test into validation and test
test_valid = ds["test"].train_test_split(test_size=0.5, seed=42)
ds["validation"] = test_valid["train"]
ds["test"] = test_valid["test"]

In [6]:
# Load pretrained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Create separate base model for comparison

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
def tokenize_fn(examples):
    en_texts = [ex["en"] for ex in examples["translation"]]
    fr_texts = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(en_texts, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(fr_texts, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = ds.map(tokenize_fn, batched=True)

In [9]:
# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
# Training arguments with W&B integration
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_bleu",
    greater_is_better=True,
    report_to="wandb",  # Enable W&B reporting
)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Load multiple metrics
bleu_metric = evaluate.load("sacrebleu")
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...


In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Post-process text
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[lbl.strip()] for lbl in decoded_labels]
    
    # Calculate all metrics
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor_metric.compute(predictions=decoded_preds, references=[l[0] for l in decoded_labels])
    bertscore_result = bertscore_metric.compute(predictions=decoded_preds, references=[l[0] for l in decoded_labels], lang="fr")
    
    # Return all metrics
    return {
        "bleu": bleu_result["score"],
        "meteor": meteor_result["meteor"],
        "bertscore_f1": np.mean(bertscore_result["f1"])
    }

In [13]:
# Log the dataset info to W&B
wandb.log({
    "dataset_name": "qanastek/ELRC-Medical-V2",
    "language_pair": "en-fr",
    "train_examples": len(tokenized_ds["train"]),
    "val_examples": len(tokenized_ds["validation"]),
    "test_examples": len(tokenized_ds["test"]),
})


In [14]:
# Evaluate base model before fine-tuning
base_trainer = Seq2SeqTrainer(
    model=base_model,
    args=training_args,
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_2049/111762798.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  base_trainer = Seq2SeqTrainer(


In [15]:
print("Evaluating base model before fine-tuning...")
base_eval_results = base_trainer.evaluate()
print("Base model metrics:", base_eval_results)

Evaluating base model before fine-tuning...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Base model metrics: {'eval_loss': 0.7314205765724182, 'eval_model_preparation_time': 0.0044, 'eval_bleu': 31.23835738621484, 'eval_meteor': 0.5108029440152249, 'eval_bertscore_f1': 0.8172544899095601, 'eval_runtime': 254.0804, 'eval_samples_per_second': 5.176, 'eval_steps_per_second': 0.327}


In [16]:
# Log base model metrics to W&B
wandb.log({
    "base_bleu": base_eval_results["eval_bleu"],
    "base_meteor": base_eval_results["eval_meteor"],
    "base_bertscore": base_eval_results["eval_bertscore_f1"]
})

In [17]:
# Setup trainer for fine-tuning with early stopping
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/tmp/ipykernel_2049/555213903.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [18]:
# Fine-tune the model with SFT
print("Starting supervised fine-tuning...")
trainer.train()

Starting supervised fine-tuning...


Epoch,Training Loss,Validation Loss,Bleu,Meteor,Bertscore F1
1,0.635400,0.549555,35.568437,0.637828,0.862817
2,0.531700,0.536628,41.962783,0.640839,0.867140
3,0.478700,0.538208,44.748357,0.662818,0.875735
4,0.385800,0.539546,43.139672,0.648396,0.869836
5,0.349600,0.540524,42.577094,0.643304,0.865908


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=3290, training_loss=0.4576783200527759, metrics={'train_runtime': 1185.7569, 'train_samples_per_second': 88.711, 'train_steps_per_second': 5.549, 'total_flos': 1001507064643584.0, 'train_loss': 0.4576783200527759, 'epoch': 5.0})

In [19]:
# Save the fine-tuned model
model_path = "./medical_translation_sft"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('./medical_translation_sft/tokenizer_config.json',
 './medical_translation_sft/special_tokens_map.json',
 './medical_translation_sft/vocab.json',
 './medical_translation_sft/source.spm',
 './medical_translation_sft/target.spm',
 './medical_translation_sft/added_tokens.json')

In [20]:
# Evaluate the fine-tuned model on the test set
print("Evaluating fine-tuned model...")
eval_results = trainer.evaluate(tokenized_ds["test"])
print("Fine-tuned model metrics:", eval_results)

Evaluating fine-tuned model...


Fine-tuned model metrics: {'eval_loss': 0.5386730432510376, 'eval_bleu': 44.56457202614395, 'eval_meteor': 0.6541183153042133, 'eval_bertscore_f1': 0.8746391176270895, 'eval_runtime': 196.9807, 'eval_samples_per_second': 6.676, 'eval_steps_per_second': 0.421, 'epoch': 5.0}


In [21]:
# Log SFT model metrics to W&B
wandb.log({
    "sft_bleu": eval_results["eval_bleu"],
    "sft_meteor": eval_results["eval_meteor"],
    "sft_bertscore": eval_results["eval_bertscore_f1"]
})

In [22]:
# Get a few examples from the test set
sample_count = 5
sample_original = ds["test"].select(range(sample_count))
sample_tokenized = tokenized_ds["test"].select(range(sample_count))

# Get predictions from the fine-tuned model
ft_predictions = trainer.predict(sample_tokenized)
ft_decoded_preds = tokenizer.batch_decode(ft_predictions.predictions, skip_special_tokens=True)

# Get predictions from the base model
base_predictions = base_trainer.predict(sample_tokenized)
base_decoded_preds = tokenizer.batch_decode(base_predictions.predictions, skip_special_tokens=True)

# Print comparisons
for i, example in enumerate(sample_original):
    english_text = example["translation"]["en"]
    ref_text = example["translation"]["fr"]
    print(f"Example {i}:")
    print("English      :", english_text)
    print("SFT prediction:", ft_decoded_preds[i])
    print("Base prediction:", base_decoded_preds[i])
    print("Reference    :", ref_text)
    print("-" * 50)
    
    # Log examples to W&B
    wandb.log({
        f"example_{i}": wandb.Table(
            columns=["English", "SFT Prediction", "Base Prediction", "Reference"],
            data=[[english_text, ft_decoded_preds[i], base_decoded_preds[i], ref_text]]
        )
    })

# Create a comparison table in W&B
comparison_table = wandb.Table(
    columns=["Example", "English", "SFT Prediction", "Base Prediction", "Reference"]
)

for i, example in enumerate(sample_original):
    english_text = example["translation"]["en"]
    ref_text = example["translation"]["fr"]
    comparison_table.add_data(
        i, english_text, ft_decoded_preds[i], base_decoded_preds[i], ref_text
    )

wandb.log({"translation_comparison": comparison_table})

Example 0:
English      : Figure 1: Measles Vaccination Coverage Rates in the EU in 2016 (Source: WHO/UNICEF JRF)
SFT prediction: Graphique 1: Taux de couverture vaccinale de la rougeole dans l'UE en 2016 (Source: CJJ de l'OMS/UNICEF)
Base prediction: Figure 1 : Taux de couverture vaccinale de la rougeole dans l'UE en 2016 (Source : OMS/UNICEF JRF)
Reference    : Figure 1: Taux de couverture vaccinale contre la rougeole dans l'UE en 2016 (Source: Formulaire commun OMS/Unicef de notification sur la vaccination)
--------------------------------------------------
Example 1:
English      : ][8: Commission Directive 2006/17/EC of 8 February 2006 implementing Directive 2004/23/EC of the European Parliament and of the Council as regards certain technical requirements for the donation, procurement and testing of human tissues and cells (OJ L38, 9.2.2006, p. 40).
SFT prediction: ][8: Directive 2006/17/CE de la Commission du 8 février 2006 portant application de la directive 2004/23/CE du Parlem